In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
# 그림 내용을 불러와서 회색으로 변경하고, 크기를 일정하게 맞추기
folder_path = "Images"
images = []
for filename in os.listdir(folder_path):
  img = cv2.imread(os.path.join(folder_path, filename))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray_resized = cv2.resize(img_gray, (256, 256))
  images.append(img_gray_resized/255.0)

# 그림을 PyTorch tensor로 변경
images_tensor = torch.tensor(images, dtype=torch.float32).unsqueeze(1) # unsqueeze로 차원을 하나 더 함 - 색깔이 회색으로 바뀌었기 때문에 색깔에 대한 차원을 맨 앞에 1개로 지정, 만약 RGB의 3개의 값이 있었다면 unsqueeze(3)이 추가되었을 것



In [ ]:
# image tensor의 차원의 숫자는 해당 tensor로 묶인 그림의 갯수, 색깔 채널의 숫자, 그림의 크기(가로? 세로?)가 순서대로
images_tensor.shape


In [ ]:
# autoencoder model을 정의
class Audoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__() # 이것은 반드시 꼭 해야 제대로 동작함
    self.encoder = nn.Sequential(
        nn.Linear(256*256, 256),
        nn.ReLU(),
        nn.Linear(256, 2)
    )
    self.decoder = nn.Sequential(
        nn.Linear(2, 256),
        nn.ReLU(),
        nn.Linear(256, 256*256),
        nn.Sigmoid()
    )
  def forward(self, x):
    x = x.view(x.size[0], -1) # 1차원으로 그림의 내용을 정리 flatten으로 같은 결과를 얻을 수 있음!
    x = self.encoder(x)
    x = self.decoder(x)
    x = x.view(x.size[0], 1, 256, 256) # 원래의 차원으로 다시 돌려주기
    return x


In [ ]:
# 모델 초기화
model = Autoencoder()

In [ ]:
# loss, optimizer 함수 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
images_tensor.shape

In [ ]:
# 훈련
num_epochs = 100
train_losses = []
for epoch in range(num_epochs):
  running_loss = 0.0
  optimizer.zero_grad()
  outputs = model(images_tensor)
  loss = criterion(outputs, images_tensor)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  train_losses.append(running_loss)
  print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, running_loss))



In [ ]:
# 훈련 과정에서 loss 값이 제대로 줄어드는지 확인 -> 만약 줄어드는 과정이 이상하거나 하면 훈련 방식 혹은 데이터 갯수 혹은 훈련의 숫자 등의 문제가 있으므로 파라미터 등을 조정하고 다시 훈련시켜야 함
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss over Epochs')
plt.show()

In [ ]:
# latent space에서의 위치를 표현해보기
with torch.no_grad():
  latent_points = model.encoder(images_tensor.view(images_tensor.size[0], -1)).numpy()

plt.scatter(latent_points[:, 0], latent_points[:, 1], c=np.arange(len(images)))
plt.xlabel('Latent Dim 1')
plt.ylabel('Latent Dim 2')
plt.title('Scatter plot of Latent Space')
plt.show()

In [ ]:
# 선택한 그림을 다시 그려보기 - 그림 재건 - 이것은 그림을 학습시키는 과정에서 매우 중요한 부분을 담당함
selected_images = images_tensor[:2]
with torch.no_grad():
  reconstructed_images = model(selected_images.view(selected_images.size[0], -1)).numpy()

# 원본 그림과 다시 복원된 그림을 비교
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
for i in range(2):
  axes[0, i].imshow(selected_images[i].squeeze(), cmap='gray')
  axes[0, i].set_title('Original Image')
  axes[1, i].imshow(reconstructed_images[i].reshape(256, 256), cmap='gray')
  axes[1, i].set_title('Reconstructed Image')

plt.show()

In [ ]:
# 선택된 두 latent space의 공간의 포인트를 interpolation?
latent1 = model.encoder(selected_images[0].view[1, -1]).detach().numpy()
latent2 = model.encoder(selected_images[1].view[1, -1]).detach().numpy()
interpolation_points = np.zeros([10, 2])
for i in range(10):
  interpolation_points[i] = latent1 + (latent2-latent1)*i/9



In [ ]:
# latent point의 위치 변화를 찍어보기
plt.scatter(interpolation_points[:, 0], interpolation_points[:, 1], color='green', label='Interpolated Points')
plt.scatter(latent_points[:, 0], latent_points[:, 1], c=np.arange(len(images)))
plt.legend()
plt.title('Latent Space with selected and interpolated points')
plt.show()


In [ ]:
with torch.no_grad():
  interpolated_images = model.decoder(torch.tensor(interpolation_points).float()).numpy()

In [ ]:
# 중간에 내적인 부분들의 그림을 그려보게 시키기
fig, axes = plt.subplots(1, 10, figsize=(20, 2))
for i in range(10):
  axes[i].imshow(interpolated_images[i].reshape(256, 256), cmap='gray')
  axes[i].axis('off')
plt.show()